<a href="https://colab.research.google.com/github/isao1974/ai_calc/blob/master/ai_calc_learn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

勉強用コード

学習済みモデルを使用する。
ph1: 加算する学習モデルを作成
ph2:四則演算する学習モデルを作成

https://qiita.com/Sa2Knight/items/221be46f2702ae453ba9
を参考に足し算を学習する学習モデルを作成する
モデルは保存する。

加算学習

In [0]:
from pandas import DataFrame
from sklearn import linear_model
import pickle

formulas = DataFrame([
    [0, 0],
    [0, 1],
    [0, 2],
    [1, 0],
    [1, 1],
    [1, 2],
    [2, 0],
    [2, 1],
    [2, 2]
])
answers = DataFrame([0, 1, 2, 1, 2, 3, 2, 3, 4])

model = linear_model.LinearRegression()
model.fit(formulas, answers)

with open('model.pickle', mode='wb') as fp:
  pickle.dump(model, fp)
  

while True:

     # 標準入力から計算式を取得
     print('> ', end='')
     inStr = input()
    
     if inStr == 'q':
      break
        
     x, y = list(map(lambda x: int(x), inStr.split(' ')))

     # 学習モデルを用いて回答を取得し、標準出力
     predected_answer = model.predict([[x, y]])
     print("{0} + {1} = {2}".format(x, y, int(predected_answer[0][0])))

保存したモデルを使う

DataFrame の１番目を演算子として学習モデルを作る。
linear_model.LinearRegression だと期待する結果が得られなかったので、
様々な識別器を試してみる。

識別器一覧
https://qiita.com/yhyhyhjp/items/3ca101f1cb5dee310c06

結果は失敗。
原因推測
* 学習データが少ない？
* モデルのハイパーパラメータが調整できてない？
* 機械学習では無理？ディープラーニングだといける？

In [0]:
from pandas import DataFrame
from sklearn import linear_model
import pickle

with open('model.pickle', mode='rb') as fp:
  model = pickle.load(fp)
  
  # 加算
  x, y = 10, 20
  predected_answer = model.predict([[x, y]])
  print("{0} + {1} = {3}".format(x, y, z, int(predected_answer[0])))

csvファイルからデータを読み込み学習させる。

csvファイルはgdrive上に用意するためgdriveをマウントする。
colaboratory 上でのCSVファイルの読み込みは下記を参照
https://qiita.com/uni-3/items/201aaa2708260cc790b8

In [0]:
from google.colab import drive
drive.mount('/content/drive')

csv ファイルからデータ読み込み

In [94]:
import pandas as pd
from pandas import DataFrame
from sklearn import linear_model
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import train_test_split
import sys
import os
import pickle

h = .02  # step size in the mesh

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    AdaBoostClassifier(),
    GaussianNB(),
    LinearDiscriminantAnalysis(),
    QuadraticDiscriminantAnalysis()]


# test file existence
testDataFile = "drive/My Drive/Colab Notebooks/data.csv"
if os.path.isfile(testDataFile) == False:
  print("file not found")
  sys.exit()
  

# read file and split data
data = pd.read_csv(testDataFile, header=0)
X = data.loc[:, ['ope', '1st', '2nd']]
Y = data.loc[:, ['ans']]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.8)


#learning
for model in classifiers:
  print(model.__class__.__name__)  
  filename = "drive/My Drive/Colab Notebooks/"
  filename += model.__class__.__name__
  filename += ".pickle"
  print(filename)
  model.fit(X_train, Y_train)
  print(model.score(X_test, Y_test))
  print("-------")
  
  with open(filename, mode='wb') as fp:
    pickle.dump(model, fp)

KNeighborsClassifier
drive/My Drive/Colab Notebooks/KNeighborsClassifier.pickle
0.8813559322033898
-------
SVC
drive/My Drive/Colab Notebooks/SVC.pickle
0.2033898305084746
-------
SVC
drive/My Drive/Colab Notebooks/SVC.pickle
0.9788135593220338
-------
DecisionTreeClassifier
drive/My Drive/Colab Notebooks/DecisionTreeClassifier.pickle
0.3305084745762712
-------
RandomForestClassifier
drive/My Drive/Colab Notebooks/RandomForestClassifier.pickle
0.5127118644067796
-------
AdaBoostClassifier
drive/My Drive/Colab Notebooks/AdaBoostClassifier.pickle


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:51: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:51: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/usr/local/lib/python3.6/dist-package

0.11864406779661017
-------
GaussianNB
drive/My Drive/Colab Notebooks/GaussianNB.pickle
0.22033898305084745
-------
LinearDiscriminantAnalysis
drive/My Drive/Colab Notebooks/LinearDiscriminantAnalysis.pickle
0.19491525423728814
-------
QuadraticDiscriminantAnalysis
drive/My Drive/Colab Notebooks/QuadraticDiscriminantAnalysis.pickle
0.0
-------


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/usr/local/lib/python3.6/dist-packages/sklearn/discriminant_

学習モデルを使って推論
90%のスコアのSVCで推論させたが、数値が小さいときはいいけど、2桁になるともうダメ。

In [99]:
from pandas import DataFrame
from sklearn import linear_model
import pickle

filename = "drive/My Drive/Colab Notebooks/"
filename += "SVC"
filename += ".pickle"


with open(filename, mode='rb') as fp:
  model = pickle.load(fp)
  
  # 加算
  ope, x, y = 0, 100, 200
  predected_answer = model.predict([[ope, x, y]])
  print("{0} ,{1}, {2} = {3}".format(ope, x, y, int(predected_answer[0])))

0 ,100, 200 = 9


deep learning で学習